# Task 2

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# importing important modules
import numpy as np
from tqdm import tqdm, trange
from matplotlib.animation import FuncAnimation, PillowWriter
from copy import deepcopy as copy

def make_gif(x, us, name, labels, skip_frame = 1, till=None, fps=25, fix_limits=True):
    """
    Create an animated GIF of magnetic field strength vs z distance over time.

    Parameters:
        x (array_like): Distance (z) values.
        us (array_like): Magnetic field strength data with shape (2, t, n) where
            2 is the number of fields, t is the number of time steps, and n is the number of distance points.
        name (str): Name of the output GIF file.
        labels (list of str): Labels for the two magnetic fields.
        skip_frame (int, optional): Skip every `skip_frame` frames in the animation. Defaults to 1.
        till (int, optional): Limit the animation to the first `till` time steps. Defaults to None.
        fps (int, optional): Frames per second of the output GIF. Defaults to 25.
        fix_limits (bool, optional): Whether to fix the y-axis limits throughout the animation. Defaults to True.
    """
    us = us[:, :till:skip_frame, :] if till is not None else us[:, ::skip_frame, :]
    max_B = np.max(us)
    min_B = np.min(us)

    p = tqdm(total=us.shape[1]+1)

    colours = [
        'tab:blue',
        'tab:orange',
    ]
    fig, ax = plt.subplots()
    def update(frame):
        p.update(1)
        ax.clear()

        ax.plot(x, us[0, frame], colours[0], label=labels[0])
        ax.plot(x, us[1, frame], colours[1], label=labels[1])
        
        ax.plot(x, us[0, 0], colours[0], label=f"Initial {labels[0]}", alpha=0.4, linestyle="--")
        ax.plot(x, us[1, 0], colours[1], label=f"Initial {labels[1]}", alpha=0.4, linestyle="--")
        if fix_limits:
            plt.ylim(min_B, max_B)
        ax.set_title(f"Magnetic Field Strength vs z Distance at Time Step {frame*skip_frame}")
        ax.set_xlabel('Distance (z)')
        ax.set_ylabel('Magnetic Field Strength (B)')
        ax.legend(loc='lower right')
        ax.grid()

    animation = FuncAnimation(fig, update, frames=us.shape[1], interval=int(1000/fps), repeat=False)
    writervideo = PillowWriter(fps=fps)
    animation.save(f"outputs/asgt2/{name}", writer=writervideo)
    p.close()

## Equations to solve:

$$ \frac{\partial B_r}{\partial t} = - \frac{\partial}{\partial z} (\alpha(z) B_\phi) + \eta_T \frac{\partial^2 B_r}{\partial z^2} $$
$$ \frac{\partial B_\phi}{\partial t} = -q\Omega B_r + \eta_T \frac{\partial^2 B_\phi}{\partial z^2} $$

Here I am supposed to solve the above equations in varying $z$ and $t$, keeping $r$ constant. Thus the $\alpha(z)=\alpha_0\sin(\pi z/h)$, $q\Omega$ and $\eta_T$ are constants in time.

$$  \frac{\partial B_r}{\partial t} = -\frac{\partial}{\partial z} (\alpha(z) B_\phi) + \eta_T \frac{\partial^2 B_r}{\partial z^2} $$

$$ \frac{\partial B_\phi}{\partial t} = -q\Omega B_r + \eta_T \frac{\partial^2 B_\phi}{\partial z^2} $$

## Crank Nicolson Method:

In this case we can take:

$$\frac{\partial U}{\partial x} = \frac{U_{i, j-1} - U_{i-1, j-1}}{\Delta x}$$
$$\frac{\partial^2 U}{\partial x^2} = \frac{1}{2\Delta x^2} \left[ (U_{i-1, j-1} + U_{i+1, j-1} - 2U_{i, j-1}) + (U_{i-1, j} + U_{i+1, j} - 2U_{i, j}) \right]$$

In the double difference term, we take the average of the two time steps.

Thus putting these in the equations:

$$  \frac{U_{i,j} - U_{i,j-1}}{\Delta t} = -\alpha(z) \frac{V_{i,j-1} - V_{i-1,j-1}}{\Delta z} + \frac{\eta_T}{2\Delta z^2} \left[ (U_{i-1,j-1} + U_{i+1,j-1} - 2U_{i,j-1}) + (U_{i-1,j} + U_{i+1,j} - 2U_{i,j}) \right] $$


$$  \frac{V_{i,j} - V_{i,j-1}}{\Delta t} = -q\Omega U_{i,j} + \frac{\eta_T}{2\Delta z^2} \left[ (V_{i-1,j-1} + V_{i+1,j-1} - 2V_{i,j-1}) + (V_{i-1,j} + V_{i+1,j} - 2V_{i,j}) \right] $$

now moving all the $j$ terms to the left side and $j-1$ terms to the right side:

$$U_{i,j} (\frac{1}{\Delta t} + \frac{\eta_T}{2\Delta z^2}) - (U_{i-1,j} + U_{i+1,j}) \frac{\eta_T}{2\Delta z^2} = U_{i,j-1} (\frac{1}{\Delta t} - \frac{1}{\Delta z^2}) - \alpha(z) \frac{V_{i,j-1} - V_{i-1,j-1}}{\Delta z} + \frac{\eta_T}{2\Delta z^2} (U_{i-1,j-1} + U_{i+1,j-1})$$

-- (1)

$$V_{i,j} (\frac{1}{\Delta t} + \frac{\eta_T}{2\Delta z^2}) - (V_{i-1,j} + V_{i+1,j}) \frac{\eta_T}{2\Delta z^2} = V_{i,j-1} (\frac{1}{\Delta t} - \frac{1}{\Delta z^2}) - q\Omega U_{i,j} + \frac{\eta_T}{2\Delta z^2} (V_{i-1,j-1} + V_{i+1,j-1})$$

-- (2)



Here to find $U_{i, j}$ we need all the values of $U$ and $V$ at time step $j-1$ (which is not a problem) along with $U_{i-1, j}$ and $V_{i-1, j}$ at time step $j$ (which is a problem). We don't have these values yet. Thus we need to construct a set of linear equations to solve this problem. Similar thing is applied to $V$ too.

Now, let us say $a = (\frac{1}{\Delta t} + \frac{1}{2\Delta z^2})$, $b = \frac{1}{2\Delta z^2}$ and $c_i = U_{i,j-1} (\frac{1}{\Delta t} - \frac{1}{\Delta z^2}) - \alpha(z) \frac{V_{i,j-1} - V_{i-1,j-1}}{\Delta z} + \frac{\eta_T}{2\Delta z^2} (U_{i-1,j-1} + U_{i+1,j-1})$ the whole left side of equation 1 (which is known). Thus we can make a matrix linear equation like $Ax=B$ where we need to get $X$ as the solution. And A and B are of the form:

$$ A = \begin{bmatrix}
 a & -b &  0 &  0 & 0 \\
-b &  a & -b &  0 & 0 \\
 0 & -b &  a & -b & 0 \\
 0 &  0 & -b &  a & -b \\
 0 &  0 &  0 & -b &  a \\
\end{bmatrix} $$

$$ B = \begin{bmatrix}
c_1 \\
c_2 \\
c_3 \\
c_4 \\
c_5 \\
\end{bmatrix} $$

Thus for every time step we need to solve this matrix equation to get the values of $U$ and $V$ at that time step.

**I haven't shown the matrix for calculating $V$ but that will be similar, only the formula for $c_i$ would change to $c_i = V_{i,j-1} (\frac{1}{\Delta t} - \frac{1}{\Delta z^2}) - q\Omega U_{i,j} + \frac{\eta_T}{2\Delta z^2} (V_{i-1,j-1} + V_{i+1,j-1})$**

In [ ]:
def solve(U, V, dt, dx, alpha0, omega, q=1, eta = 1, verbose = True):
    assert U.shape == V.shape, "U and V must have the same shape/resolution"
    U = -copy(U)  # copy to avoid changing the original
    V = copy(V)  # copy to avoid changing the original
    # if verbose: print(f"U shape: {U.shape}, V shape: {V.shape}")
    a = 1/dt + eta/(dx**2)
    b = eta/(2*dx**2)
    if verbose:
        print(f"Dynamo number = {alpha0*q*omega*1/eta**3}")
        print(f"a: {a}, b: {b}")

    A = np.zeros((U.shape[1]-2, U.shape[1]-2))
    for row in range(A.shape[0]):
        A[row, row] = a
        if row > 0:
            A[row, row-1] = -b
        if row < A.shape[0]-1:
            A[row, row+1] = -b

    A_inv = np.linalg.inv(A)

    if verbose: ra = trange
    else: ra = range

    z = np.linspace(-1, 1, U.shape[1]-1)

    for j in ra(1, U.shape[0]):  # for all the time steps
        # make B for U
        # print(z[:-1].shape, (V[:, 1:-1] - V[:, :-2]).shape)
        BU = (
              U[j-1, 1:-1]*(1/dt - eta/dx**2)
            # - alpha0*np.sign(z[:-1])*(V[j-1, 1:-1] - V[j-1, :-2])/dx
            - alpha0*np.sin(z[:-1]*np.pi)*(V[j-1, 1:-1] - V[j-1, :-2])/dx
            + b*(U[j-1, :-2] + U[j-1, 2:])
        )
        # print(BU.shape, U.shape)
        U[j, 1:-1] = A_inv @ BU

        BV = ( 
              V[j-1, 1:-1]*(1/dt - eta/dx**2) 
             - omega*q*U[j, 1:-1] 
             + b*(V[j-1, :-2] + V[j-1, 2:]) 
        )
        V[j, 1:-1] = A_inv @ BV.T

    return -U, V

## Repeat the investigation you had done for task 1, with the new equations, for different values of the dynamo number.

In [ ]:
# Defining Grid and Initial Conditions
T = 100
dT = 0.1
NT = int(T/dT)
L = 2
dL = 0.01
NL = int(L/dL)
U = np.zeros((NT, NL))  # Initialize U matrix
V = np.zeros((NT, NL))  # Initialize V matrix

U[0, :] = -np.sin(np.linspace(0, np.pi, NL))  # Set the first row of U to sin(x)
V[0, :] = np.sin(np.linspace(0, np.pi, NL))

# np.random.seed(0)
# U[0, :] = 1*(np.random.rand(NL)-0.5)
# V[0, :] = 1*(np.random.rand(NL)-0.5)

U, V = solve(U, V, dT, dL, alpha0=1, omega = 9, q = -1, eta = 1)

make_gif(
    np.linspace(-L/2, L/2, NL),
    np.array([U, V]),
    "growing_dynamo.gif",
    labels=["$B_r$", "$B_\phi$"],
    skip_frame=10,
    till=None,
    fps=24,
    fix_limits=False
)

In [ ]:
i = int(L/(2*dL))
plt.plot(np.log(np.abs(U[:, int(L/(2*dL))])), label = "$B_r$")
plt.plot(np.log(np.abs(V[:, int(L/(2*dL))])), label = "$B_\phi$")

plt.grid()
plt.xlabel("Time Step")
plt.ylabel("Log Magnetic Field Strength")
plt.title("Log Magnetic Field Strength vs Time Step at Center of the Dynamo")
plt.legend()

plt.savefig("outputs/asgt2/growing_dynamo_log_plot.png")

We can see that for Dynamo number $D = -7$ we get a growing dynamo. The gif is shown below:

![growing_dynamo.gif](outputs/asgt2/growing_dynamo.gif)

And here is the plot of the log of the magnetic field with respect to time at the center of the space:

![growing_dynamo.png](outputs/asgt2/growing_dynamo_log_plot.png)

In [ ]:
# Defining Grid and Initial Conditions
T = 10
dT = 0.05
NT = int(T/dT)
L = 2
dL = 0.01
NL = int(L/dL)
U = np.zeros((NT, NL))  # Initialize U matrix
V = np.zeros((NT, NL))  # Initialize V matrix

U[0, :] = -np.sin(np.linspace(0, np.pi, NL))  # Set the first row of U to sin(x)
V[0, :] = np.sin(np.linspace(0, np.pi, NL))

U, V = solve(U, V, dT, dL, alpha0=1, omega = 7.5, q = -1, eta = 1)

# make_gif(np.linspace(-L/2, L/2, NL), np.array([U, V]), "decaying_dynamo.gif", labels=["$B_r$", "$B_\phi$"], skip_frame=1, till=None, fps=24)

In [ ]:
i = int(L/(2*dL))
plt.plot(np.log(np.abs(U[:, int(L/(2*dL))])), label = "$B_r$")
plt.plot(np.log(np.abs(V[:, int(L/(2*dL))])), label = "$B_\phi$")

plt.grid()
plt.xlabel("Time Step")
plt.ylabel("Log Magnetic Field Strength")
plt.title("Log Magnetic Field Strength vs Time Step at Center of the Dynamo")
plt.legend()

plt.savefig("outputs/asgt2/decaying_dynamo_log_plot.png")

We can see that for Dynamo number $D = -6$ we get a decaying dynamo. The gif is shown below:

![growing_dynamo.gif](outputs/asgt2/decaying_dynamo.gif)

And here is the plot of the log of the magnetic field with respect to time at the center of the space:

![growing_dynamo.png](outputs/asgt2/decaying_dynamo_log_plot.png)

## Finding $D_c$

We know that at $D > D_c$ the dynamo grows exponentially and at $D < D_c$ the dynamo decays exponentially. Thus we need to define the notion of growth and decay. Let me define the growth and decay as follows:

In [ ]:
def growth_rate(U, V, point=None, after=10):
    if point is None:
        point = U.shape[1]//2
    
    U = U[after:, point]
    V = V[after:, point]
    x = np.linspace(0, U.shape[0]*1e-6, U.shape[0])
    
    # print(U.shape, V.shape, x.shape)
    
    # print("Calculating Growth Rate")
    # print(U.shape, V.shape)
    
    Um, Uc = np.polyfit(x, np.log(np.abs(U)), 1)
    Vm, Vc = np.polyfit(x, np.log(np.abs(V)), 1)
    # print(Um, Vm)
    
    return sum((Um, Vm))

We could have used advanced techniques like **gradient descent** to find the value of $D_c$ but I am going to use a simple technique. I am going to plot the values of `growth_rate` for a range of $D$ values and find the value of $D$ where the `growth_rate` is 1. This is the value of $D_c$.

We have seen that for $D = 0.1$ the dynamo grows and for $D = 0.01$ the dynamo decays. Thus the value of $D_c$ should be between 0.01 and 0.1. Thus I am going to take a range of $D$ values from 0.01 to 0.1 and find the value of $D$ where the `growth_rate` is 1.

In [ ]:
T = 10
dT = 0.05
NT = int(T/dT)
L = 2
dL = 0.01
NL = int(L/dL)
U = np.zeros((NT, NL))  # Initialize U matrix
V = np.zeros((NT, NL))  # Initialize V matrix

U[0, :] = -np.sin(np.linspace(0, np.pi, NL))  # Set the first row of U to sin(x)
V[0, :] = np.sin(np.linspace(0, np.pi, NL))

Dlow = 7.5
Dhigh = 9
N = 100
omegas = np.linspace(Dlow, Dhigh, N)
rates = []

for D in tqdm(omegas):
    U, V = solve(U, V, dT, dL, alpha0=1, omega = D, q = -1, eta = 1, verbose=False)
    # print(U.shape, V.shape)

    rate = growth_rate(U, V)
    rates.append(rate)
rates = np.array(rates)

In [ ]:
def get_dc(omegas, rates):
    m, c = np.polyfit(omegas, rates, 1)
    # get the point where rate is 0
    return -c/m  # if mx+c = 0, then x = -c/m

In [ ]:
Dc = get_dc(-omegas, rates)
plt.plot(-omegas, rates, label="Growth Rate")
plt.grid()
plt.axvline(Dc, color='r', linestyle='--', label=f"$D_c = {Dc:.4f}$")
plt.axhline(0, color='g', linestyle='--', label=f"growth rate = 1")
plt.xlabel("D")
plt.ylabel("Growth Rate")
plt.legend()
plt.title("Growth Rate vs Diffusion Constant")
plt.text(Dc, 0, f"$D_c = {Dc:.4f}$", fontsize=12, color='k', ha='right', va='top')
plt.savefig("outputs/asgt2/find_Dc.png")  

![find_Dc.png](outputs/asgt2/find_Dc.png)

Simulation of the dynamo at $D = D_c$

In [ ]:
# Defining Grid and Initial Conditions
T = 10
dT = 0.05
NT = int(T/dT)
L = 2
dL = 0.01
NL = int(L/dL)
U = np.zeros((NT, NL))  # Initialize U matrix
V = np.zeros((NT, NL))  # Initialize V matrix

U[0, :] = -np.sin(np.linspace(0, np.pi, NL))  # Set the first row of U to sin(x)
V[0, :] = np.sin(np.linspace(0, np.pi, NL))

U, V = solve(U, V, dT, dL, alpha0=1, omega = -Dc, q = -1, eta = 1)

# make_gif(np.linspace(-L/2, L/2, NL), np.array([U, V]), "stable_dynamo.gif", labels=["$B_r$", "$B_\phi$"], skip_frame=1, till=None, fps=24)

In [ ]:
i = int(L/(2*dL))
plt.plot(np.log(np.abs(U[:, int(L/(2*dL))])), label = "$B_r$")
plt.plot(np.log(np.abs(V[:, int(L/(2*dL))])), label = "$B_\phi$")

plt.grid()
plt.xlabel("Time Step")
plt.ylabel("Log Magnetic Field Strength")
plt.title("Log Magnetic Field Strength vs Time Step at Center of the Dynamo")
plt.legend()

plt.savefig("outputs/asgt2/stable_dynamo_log_plot.png")

Here we can see the stable dynamo created at the calculated value of $D_c$:

![growing_dynamo.gif](outputs/asgt2/stable_dynamo.gif)

And here is the plot of the log of the magnetic field with respect to time at the center of the space:

![growing_dynamo.png](outputs/asgt2/stable_dynamo_log_plot.png)